In [ ]:
import random
import numpy as np

class NN:
    def __init__(self, layers, weights_init=default_init):
        self.weights = weights_init(layers)
        self.bias = bias_init(layers)


    def default_init(layers):
        np.random.normal()


        return weights


    